# Lid Driven Cavity Flow

Equations:

$$
\begin{align} 
    &\rho \frac{\partial u}{\partial t} + \rho \left(u\frac{\partial u}{\partial x } + v \frac{\partial u}{\partial y }\right) + \frac{\partial p}{\partial x } - \mu \left( \frac{\partial^2 u}{\partial x^2 } + \frac{\partial^2 u}{\partial y^2 } \right)  = 0 \\
    &\rho \frac{\partial v}{\partial t} + \rho \left(u\frac{\partial v}{\partial x } + v \frac{\partial v}{\partial y }\right) + \frac{\partial p}{\partial y } - \mu \left( \frac{\partial^2 v}{\partial v^2 } + \frac{\partial^2 v}{\partial y^2 } \right)  = 0 \\
    &\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0\\
    &\Omega = [0,1]\times[0,1], \quad 0 \leq t \leq 1.\\
    &Re = \frac{\rho u L}{\mu}, \quad 1. \leq Re \leq 50.\\
    &u(x,1,t) = w(1-(2x-1)^6) t,\\
    &u(0,y,t) = u(1,y,t) = u(x,0,t) = 0. \\
    &v(0,y,t) = v(1,y,t) = v(x,0,t) = v(x,1,t) = 0.\\
    &u(x,y,0) = v(x,y,0) = 0.\\
    &p(0,0,t) = 0.
\end{align}
$$

In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES']='-1'
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['export OPENBLAS_NUM_THREADS']='16'

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import fenics as fe
import numpy as np
import matplotlib.pyplot as plt

import time
import gc

from LFM.utilities import *


In [ ]:
plt.rcParams.update({
"text.usetex": True,
"font.family": "sans-serif",
"font.sans-serif": ["Helvetica"],
'axes.labelsize': 16,
'axes.titlesize': 16,
'xtick.labelsize' : 16,
'ytick.labelsize' : 16
        })
# latex font definition
plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Computer Modern Roman']})
          

In [ ]:
Data = 'GT'
# Data = 'Ob'
# Data = 'Plotting_gt'

if Data == 'GT':
    dirname='data-groundTruth-nslid_100_0_8-1--0_1-1--1-1'
if Data == 'Ob':
    dirname='data-ObsData-nslid_100'
    
setBaseParams = False

if Data == 'Plotting_gt':
    dirname='data-plotting_Re10'
#     dirname='data-plotting_Re1'
    setBaseParams = True
os.makedirs(dirname, exist_ok=True)

SAVE = True
PLOT = False

In [ ]:
def solve_lid_driven_cavity_flow(params, times, w_val=1., nx=100,ny=100, dt=0.01, T=1, require_plots = (True, 10)):
    # get parameters
    rho, mu = params
    
    mesh = fe.UnitSquareMesh(nx,ny)
    
    # Define function spaces (P2-P1)
    V = fe.VectorFunctionSpace(mesh, "Lagrange", 2)
    Q = fe.FunctionSpace(mesh, "Lagrange", 1)

    # Define trial and test functions
    u = fe.TrialFunction(V)
    p = fe.TrialFunction(Q)
    v = fe.TestFunction(V)
    q = fe.TestFunction(Q)
    
    # Define time-dependent boundary velocity at y=1
    u_top = fe.Expression(("(( - 1. * pow( ( 2*x[0] -1 ), 6) + 1. ) * t )*{}".format(w_val), "0"), t=0.0, degree=16)
    
    # set up bcs
    top_cond = fe.DirichletBC(V, u_top,
                       "on_boundary && \
                       x[1] > 1.0 - DOLFIN_EPS")

    bc_p = fe.DirichletBC(Q, 0., "x[0] < DOLFIN_EPS && x[1] < DOLFIN_EPS", "pointwise")
    
    noslip = fe.DirichletBC(V, (0, 0),
                         "on_boundary && \
                         x[1] <= 1.0 - DOLFIN_EPS")

    bcu = [top_cond, noslip]
    bcp = [bc_p]
    
    # Create functions
    u0 = fe.Function(V)
    u1 = fe.Function(V)
    p1 = fe.Function(Q)

    # Define coefficients
    k = fe.Constant(dt)
    ρ = fe.Constant(rho)
    μ = fe.Constant(mu)
    
    # Tentative velocity step
    F1 = (ρ/k)*fe.inner(u - u0, v)*fe.dx + ρ*fe.inner(fe.grad(u0)*u0, v)*fe.dx + \
         μ*fe.inner(fe.grad(u), fe.grad(v))*fe.dx

    a1 = fe.lhs(F1)
    L1 = fe.rhs(F1)

    # Pressure update
    a2 = fe.inner(fe.grad(p), fe.grad(q))*fe.dx
    L2 = -(1/k)*fe.div(u1)*q*fe.dx

    # Velocity update
    a3 = fe.inner(u, v)*fe.dx
    L3 = fe.inner(u1, v)*fe.dx - k*fe.inner(fe.grad(p1), v)*fe.dx
    
    # Assemble matrices
    A1 = fe.assemble(a1)
    A2 = fe.assemble(a2)
    A3 = fe.assemble(a3)
    
    # Use amg preconditioner if available
    prec = "amg" if fe.has_krylov_solver_preconditioner("amg") else "default"
    
    # Time-stepping
    t = dt
    velocity_sols = [u1.copy(deepcopy=True)]
    p_sols = [p1.copy(deepcopy=True)]
    t_sols = [0.0]
    for t in times:
        
        # update top boundary condition
        u_top.t = t

        # Compute tentative velocity step
        b1 = fe.assemble(L1)
        [bc.apply(A1, b1) for bc in bcu]
        fe.solve(A1, u1.vector(), b1, "gmres", "default")
        fe.end()

        # Pressure correction
        b2 = fe.assemble(L2)
        [bc.apply(A2, b2) for bc in bcp]
        fe.solve(A2, p1.vector(), b2, "gmres", prec)
        fe.end()

        # Velocity correction
        b3 = fe.assemble(L3)
        [bc.apply(A3, b3) for bc in bcu]
        fe.solve(A3, u1.vector(), b3, "gmres", "default")
        fe.end()

        # save to lists
        velocity_sols.append(u1.copy(deepcopy=True))
        p_sols.append(p1.copy(deepcopy=True))
        t_sols.append(t)

        # Move to next time step
        u0.assign(u1)
        t += dt
    
    
    vel_grid = V.tabulate_dof_coordinates()
    p_grid = Q.tabulate_dof_coordinates()

    velocity_x_sols = []
    velocity_y_sols = []
    for velocity in velocity_sols:
        velocity_x, velocity_y = velocity.split()
        velocity_x_sols.append(velocity_x.copy(deepcopy=True))
        velocity_y_sols.append(velocity_y.copy(deepcopy=True))
        
    results = {
        'vel_grid': vel_grid,
        'p_grid': p_grid,
        'velocities': velocity_sols,
        'pressures': p_sols,
        'times': t_sols,
        'u': velocity_x_sols,
        'v': velocity_y_sols
    }
    return results

In [ ]:
def get_solution_array(sol):
    return sol.vector().get_local()

def get_sol_at_locs(f, locs):
    """
    Evaluate fenics function f at the locations given in list/array locs (each row of locs is
    a single location)
    """
    return np.array([f(fe.Point(loc)) for loc in locs])

In [ ]:
if False:
    rho = 2.
    mu  = 0.1
    w   = 2.
    n   = 250

    params = [rho, mu]
    nt     = 100
    times  = np.linspace(0., 1., nt) 
    Re     = rho * w * 1. / mu

    print('Re = ', Re)

    startTime =  time.time()
    results   = solve_lid_driven_cavity_flow(params, times, w_val=w, nx=n, ny=n)
    endTime   = time.time()
    total_time = endTime - startTime
    print('total_time ', total_time)

    vel_grid = results['vel_grid']
    p_grid = results['p_grid']

    us = results['u']
    vs = results['v']
    ps = results['pressures']
    ts = results['times']

    i = len(us)-1
    u = us[i]
    v = vs[i]
    p = ps[i]

    nxp, nyp = 100, 100
    x, y   = tf.linspace(0., 1., nxp), tf.linspace(0., 1., nyp)
    X, Y   = np.meshgrid(x, y, indexing='xy')
    Xr, Yr = tf.reshape(X, [-1, 1]), tf.reshape(Y, [-1, 1])
    XYr    = tf.concat([Xr, Yr], 1)

    p_vec = get_sol_at_locs(p, XYr)
    u_vec = get_sol_at_locs(u, XYr)
    v_vec = get_sol_at_locs(v, XYr)

    P_vec = tf.reshape(p_vec, [nxp,nyp])
    U_vec = tf.reshape(u_vec, [nxp,nyp])
    V_vec = tf.reshape(v_vec, [nxp,nyp])

    fig, ax = plt.subplots()

    mag_vec = tf.linalg.norm(tf.concat([U_vec[...,None], V_vec[...,None]],-1), axis=-1)
    mag_vec = np.array(mag_vec)

    print('mag_vec.shape = ', mag_vec.shape)
    print('X.shape = ', X.shape)

    strm  = ax.streamplot(X, Y, U_vec, V_vec, color=mag_vec, density=6,cmap = plt.cm.viridis)
    ax.contourf(X,Y, P_vec, 50)
    fig.colorbar(strm.lines, ax=ax)
    ax.set(xlabel='x', ylabel='y', title='Stream plot')
    ax.set_xlim(0., 1.)
    ax.set_ylim(0., 1.)

    plt.show()

    print('max u = ', tf.reduce_max(u_vec))
    print('max v = ', tf.reduce_max(v_vec))
    print('max p = ', tf.reduce_max(p_vec))

    print('mean u = ', tf.reduce_mean(u_vec))
    print('mean v = ', tf.reduce_mean(v_vec))
    print('mean p = ', tf.reduce_mean(p_vec))

In [ ]:
dimZ = 2
dimW = 1

zDist = tfd.Uniform(low=tf.constant([0.8, 0.1], shape=[dimZ]), high=tf.constant([1., 1.], shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant([1.], shape=[dimW]), high=tf.constant([1.], shape=[dimW]))


nSamples  = 25
allTrueZs = zDist.sample(nSamples)
allTrueWs = wDist.sample(nSamples)


nx_grid, ny_grid, nt_grid = 200, 200, 100
nx_gt, ny_gt              = 50, 50

times  = np.linspace(0., 1., nt_grid) 
X, Y   = np.meshgrid( tf.linspace(0., 1., nx_gt), tf.linspace(0., 1., ny_gt), indexing='xy')
Xr, Yr = tf.reshape(X, [-1,1]), tf.reshape(Y, [-1,1])
XYr    = tf.concat([Xr, Yr], 1)

T      = range(0, nt_grid, 2) 

Tsln   = times[T]
print('Tsln\n', Tsln)

prepend   = ''
startTimeAll = time.time()
totalTimeFE  = 0.

if Data == 'GT' or Data == 'Plotting_gt':

    P_vec_all = []
    U_vec_all = []
    V_vec_all = []
    
    XY = tf.concat([X[None,...], Y[None,...]],0)
    if SAVE:
        np.savetxt(dirname + '/' + prepend + 'allZ.dat', allTrueZs )
        np.savetxt(dirname + '/' + prepend + 'allW.dat', allTrueWs )
        np.save(dirname + '/' + prepend + 'FenicsMeshXY', XY, allow_pickle=False )
        np.save(dirname + '/' + prepend + 'FenicsMeshT', np.array(Tsln), allow_pickle=False )

elif Data == 'Ob':
    numObs        = 100 
    allRandomXTs  = []
    noisySlns     = []
    if SAVE:
        np.savetxt(dirname+'/' +prepend+'allZ.dat'   , allTrueZs )
        np.savetxt(dirname+'/' +prepend+'allW.dat'   , allTrueWs )

    sigma_y = tf.constant(0.05)

for i in range(allTrueZs.shape[0]):
    
    z_i =  np.array(allTrueZs[i]).squeeze()
    w_i =  allTrueWs[i,0]
    
    if setBaseParams == True:
        z_i =  np.array([1., 0.1])
        w_i =  1.
        
    print('z_i', z_i)
    print('w_i', w_i)
        
    #         rho         mu
    params = [z_i[0], z_i[1]]
    nt     = 100
    
    Re     = params[0] * w_i * 1. / params[1]

    P_vec_times = []
    U_vec_times = []
    V_vec_times = []

    print('Re = ', Re)

    startTime  =  time.time()
    results    = solve_lid_driven_cavity_flow(params, times, w_val=w_i, nx=nx_grid, ny=ny_grid)
    endTime    = time.time()
    total_time = endTime - startTime
    print('total_time ', total_time)
    totalTimeFE += total_time

    us = results['u']
    vs = results['v']
    ps = results['pressures']
    ts = results['times']

    for j_time in T:

        u = us[j_time]
        v = vs[j_time]
        p = ps[j_time]

        p_vec = get_sol_at_locs(p, XYr)
        u_vec = get_sol_at_locs(u, XYr)
        v_vec = get_sol_at_locs(v, XYr)

        P_vec = tf.reshape(p_vec, [nx_gt,ny_gt])
        U_vec = tf.reshape(u_vec, [nx_gt,ny_gt])
        V_vec = tf.reshape(v_vec, [nx_gt,ny_gt])

        P_vec = tf.reshape(p_vec, [nx_gt,ny_gt])
        U_vec = tf.reshape(u_vec, [nx_gt,ny_gt])
        V_vec = tf.reshape(v_vec, [nx_gt,ny_gt])

        P_vec_times.append(P_vec)
        U_vec_times.append(U_vec)
        V_vec_times.append(V_vec)

    if Data == 'GT':

        P_vec_all.append(P_vec_times)
        U_vec_all.append(U_vec_times)
        V_vec_all.append(V_vec_times)


    if Data == 'Ob':

        randomXIndex = tf.random.uniform(minval = 0, maxval = nx_grid-1, shape=[numObs], dtype=tf.int32)
        randomYIndex = tf.random.uniform(minval = 0, maxval = ny_grid-1, shape=[numObs], dtype=tf.int32)
        randomTIndex = tf.random.uniform(minval = 0, maxval = nt_grid-1, shape=[numObs], dtype=tf.int32)

        allRandomXTs.append( tf.concat([X[randomXIndex, randomYIndex, randomTIndex][:, None], 
                                        Y[randomXIndex, randomYIndex, randomTIndex][:, None],
                                        T[randomXIndex, randomYIndex, randomTIndex][:, None]], 1) )

        subsample_sols = sols[randomXIndex, randomYIndex, randomTIndex] 


    print('\n\nSample {}\n\n'.format(i))

    gc.collect()
    if PLOT:
        for i_time in range(len(T)):
            print('plotting')

            plt.contourf(X, Y, P_vec_times[i_time], 50, cmap=plt.cm.viridis)
            plt.ylabel(r'$y$')
            plt.xlabel(r'$x$')
            plt.colorbar()
            plt.tight_layout()
            plt.savefig(dirname+'/'+'contourP{}time{}.png'.format(i, i_time), dpi=300)
            plt.show()  
            plt.close()

            plt.contourf(X, Y, U_vec_times[i_time], 50, cmap=plt.cm.viridis)
            plt.ylabel(r'$y$')
            plt.xlabel(r'$x$')
            plt.colorbar()
            plt.tight_layout()
            plt.savefig(dirname+'/'+'contourU{}time{}.png'.format(i, i_time), dpi=300)
            plt.show()  
            plt.close()


            plt.contourf(X, Y, V_vec_times[i_time], 50, cmap=plt.cm.viridis)
            plt.ylabel(r'$y$')
            plt.xlabel(r'$x$')
            plt.colorbar()
            plt.tight_layout()
            plt.savefig(dirname+'/'+'contourV{}time{}.png'.format(i, i_time), dpi=300)
            plt.show()  
            plt.close()

            fig, ax = plt.subplots()

            mag_vec = tf.linalg.norm(tf.concat([U_vec_times[i_time][...,None], V_vec_times[i_time][...,None]],-1), axis=-1)
            mag_vec = np.array(mag_vec)
            
            density   = 8
            linewidth = 0.1
            cmap      = plt.cm.seismic
            strm  = ax.streamplot(X, Y, U_vec_times[i_time], V_vec_times[i_time], color=mag_vec, density=density,cmap = cmap,
                        broken_streamlines=False, linewidth=linewidth, arrowsize=0.)       
            fig.colorbar(strm.lines, ax=ax)
            ax.set(xlabel=r'$x$', ylabel=r'$y$')
            ax.set_xlim(0., 1.)
            ax.set_ylim(0., 1.)
            fig.tight_layout()
            fig.savefig(dirname+'/'+'contourStreamLines{}time{}.png'.format(i, i_time), dpi=300)
            plt.show()
            plt.close()
            
if Data == 'GT':
    if SAVE:
        np.save(dirname+'/' +prepend+'SolutionsPGT'   , np.array(P_vec_all),  allow_pickle=False ) 
        np.save(dirname+'/' +prepend+'SolutionsUGT'   , np.array(U_vec_all),  allow_pickle=False ) 
        np.save(dirname+'/' +prepend+'SolutionsVGT'   , np.array(V_vec_all),  allow_pickle=False ) 
        with  open(dirname+"allInfoRun.txt", "w") as f:
             f.write('time taken all sims = {:.2f}m\n'.format( time.time() - startTimeAll ), 's')
        f.close()

print('time taken all sims= ', time.time() - startTimeAll, 's')
print('total time FE = ', totalTimeFE, 's')
if PLOT: plt.show()
